# TOC entitlements mapped

In [1]:
import intake
import geopandas as gpd
import pandas as pd

import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps, WidgetControl
from ipywidgets import link, FloatSlider, Text, HTML
from ipywidgets.embed import embed_minimal_html
import branca.colormap
import json

catalog = intake.open_catalog('../catalogs/*.yml')

In [ ]:
def make_tract_level(start_date):    
    # Import data
    pcts = catalog.pcts2.read()
    parents = catalog.pcts_parents.read()
    crosswalk_parcels_tracts = catalog.crosswalk_parcels_tracts.read()
    
    # Subset PCTS, we'll start with last 10 years
    pcts = pcts[pcts.CASE_FILE_DATE >= start_date]
    
    # (1a) Only keep parent cases
    m1 = pd.merge(pcts, parents, on = 'PARENT_CASE', how = 'inner', validate = 'm:1')

    # (1b) Make cases parcel-level
    parents_by_parcel = (m1.groupby(['AIN'])
                         .agg({'PARENT_CASE':'count'})
                         .reset_index()
                         .rename(columns = {'PARENT_CASE':'num_cases'})
                        )

    # (1c) Merge in tract info and aggregate to tract-level
    m2 = pd.merge(parents_by_parcel, crosswalk_parcels_tracts, on = 'AIN', how = 'inner', validate = '1:1')

    # Even though the column num_AIN shows there are some parcels with more than 1 obs,
    # once we merged in parent cases, no AIN shows up more than once

    # (1d) Subset by prefixes?
    def subset_by_prefix_suffix(row):
        if any(p in row.CASE_NBR for p in prefix_or_suffix_list):
            return True
        else: 
            return False 
    
    ent_by_tract = m2.groupby(['GEOID', 'pop']).agg({'num_cases':'sum', 'AIN':'count'}).reset_index()
    
    
    # (2a) Only keep suffixes
    suffix = m1.loc[:, '1A':'ZV']
    
    m3 = pd.merge(m1[['AIN']], suffix, left_index = True, right_index = True)

    # (2b) Make suffixes parcel-level
    suffix_by_parcel = (m3.pivot_table(index = 'AIN', aggfunc = 'sum')
                        .reset_index()
                       )

    # (2c) Merge in tract info and aggregate to tract-level
    m4 = pd.merge(suffix_by_parcel, crosswalk_parcels_tracts, 
                                 on = 'AIN', how = 'left', validate = '1:1')

    # Aggregate the number of suffixes by tract
    suffix_by_tract = m4.pivot_table(index = ['GEOID', 'pop'], aggfunc = 'sum').reset_index()
    
    
    # (3) Merge number of cases and suffixes by tract
    df = pd.merge(ent_by_tract, suffix_by_tract, on = ['GEOID', 'pop'], how = 'left', validate = '1:1')
        
    # (4) Make sure everything returns as integers and not floats
    colnames = list(df.columns)

    for r in ['GEOID', 'pop', 'AIN']:
        colnames.remove(r)
    
    # Count number of suffixes, since each ENT can have any number of suffixes involved
    df[colnames] = df[colnames].fillna(0).astype(int) 
    df['num_suffix'] = df[colnames].sum(axis=1)
        
    return df

In [ ]:
start_date = "2017-10"
pcts = make_tract_level(start_date)

# Just keep TOC ent for the tract
keep = ['GEOID', 'pop', 'num_cases', 'AIN', 'num_AIN', 'parcel_tot', 'parcelsqft', 'TOC',]
pcts = pcts[keep]

In [ ]:
# Prep census tract data to make map
census = catalog.census_tracts.read()
census = (census[['GEOID10', 'geometry']]
        .assign(num = 0)
        .rename(columns = {'GEOID10':'GEOID'})
         ).to_crs("EPSG:4326")

In [ ]:
# Make a map showing TOC ent by tract, but also show all census tracts in LA
gdf = pd.merge(census.drop(columns = 'num'), pcts, on = 'GEOID', how = 'inner', validate = '1:1')
gdf = gdf[gdf.TOC > 0]

In [ ]:
geo_data = json.loads(gdf.set_index('GEOID').to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
df = dict(zip(gdf['GEOID'].tolist(), gdf['TOC'].tolist()))


census_geo = json.loads(census.set_index('GEOID').to_json())
census_df = dict(zip(census['GEOID'].tolist(), census['num'].tolist()))

In [ ]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 11,
                  basemap = basemaps.CartoDB.Positron)

census_tracts = ipyleaflet.Choropleth(
    geo_data = census_geo,
    choro_data = census_df, 
    colormap = branca.colormap.LinearColormap([
        "#FFFFFF",
        "#FFFFFF",        
    ]),
    border_color = '#999999',
    style = {'fillOpacity': 0.4, 'weight': 0.5, 'color': '#999999', 'opacity': 0.8},
    value_min = 0,
    value_max = 0,
    name = 'Census Tracts'
)

toc = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = df,
    colormap = branca.colormap.LinearColormap([
        "#C9FFFD",
        "#06BEE1",
        "#1768AC",
        "#2541B2",
        "#03256C"
    ]),
    border_color = 'white',
    style = {'fillOpacity': 0.8, 'weight': 0.5, 'color': 'white', 'opacity': 0.8},
    hover_style = {'fillOpacity': 0.85},
    value_min = 0,
    value_max = 60,
    name = 'TOC entitlements'
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def click_handler(event = None, id = None, properties = None):
    label.value = properties['TOC']

def update_html_tract(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Number of TOC Entitlements (2017-2019):
        {} 
    '''.format(id, feature['properties']['TOC'])    
    
toc.on_click(click_handler) 
toc.on_hover(update_html_tract)

m.add_layer(census_tracts)
m.add_layer(toc)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
layers_control = ipyleaflet.LayersControl(position = 'topright')

m.add_control(control)
m.add_control(layers_control)

m.layout.height = '100%'
m.layout.min_height = '600px'

m